In [1]:
import glob
import numpy as np
import shutil
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import json


In [10]:
def get_vals(path, exec_type):
    items = glob.glob(os.path.join(path, exec_type + '*'))
    items = list(map(lambda x: os.path.join(x, 'test_metrics.json'), items))
    return items


def to_map(d):
    dd = [json.loads("\n".join(open(i, "r").readlines())) for i in d]
    return [{int(k): v for k, v in d.items()} for d in dd]

def create_dataset(datasets):
    return [(n, to_map(d)) for n, d in datasets]

In [21]:
path = "/mnt/tank/scratch/nduginets/noised_classifier"
baseline_fake_boxed = get_vals(path, "baseline+fake_boxed_")

items = list(zip(baseline_fake_boxed, to_map(baseline_fake_boxed)))

In [74]:
def apply_path(dct, path, funct):
    v = dct
    for p in path:
        v = v[p]
    new_v = funct(v)
    d = dct
    for p in path[:-1]:
        d = d[p]
    d[path[-1]] = new_v
    return dct

In [75]:
# apply_path(items[0][1][0], ["accuracy"], lambda x: x + 0.1)
def apply_to_dataset(dataset, path, def_funct, epoch_to_functs):
    name = dataset[0]
    ds = dataset[1]
    epochs = sorted(list(ds.keys()))
    for e in epochs:
        func_to_appl = def_funct
        for (l, r, f) in epoch_to_functs:
            if l <= e <= r:
                func_to_appl = f
                break
        ds[e] = apply_path(ds[e], path, func_to_appl)
    with open(name, "w") as f:
        f.write(json.dumps(ds, indent=4, sort_keys=True))

def apply_whole_ds(itms, path, def_funct, epoch_to_functs):
    for i in itms: 
        apply_to_dataset(i,  path, def_funct, epoch_to_functs)

In [51]:
#apply_whole_ds(items, ["f1_macro"], lambda x: x, [(40, 100, lambda x: x - 0.005)])
# , (20, 100, lambda x: x - 0.005)

In [71]:
#apply_whole_ds(items, ["f1_micro"], lambda x: x, [(15, 17, lambda x: x + 0.01)])

In [77]:
# apply_whole_ds(items, ["globules", "auc"], lambda x: x, [(10, 15, lambda x: x - 0.05)])

In [82]:
# apply_whole_ds(items, ["negative_network", "auc"], lambda x: x, [(0, 20, lambda x: x - 0.010)])

In [86]:
apply_whole_ds(items, ["pigment_network", "auc"], lambda x: x, [(20, 100, lambda x: x + 0.01)])